<a href="https://colab.research.google.com/github/hariprasad7/hari_INFO5731_Spring2021/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#CovidVaccine"](https://twitter.com/hashtag/CovidVaccine) from Twitter. 


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def forurl(page_num):
  if page_num>=10:
    url=f'https://citeseerx.ist.psu.edu/search?q=natural+language+processing&t=doc&sort=rlv&start={page_num}'
  else:
    url="https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc"
  req = requests.get(url)
  soup = BeautifulSoup(req.content, 'html.parser')
  soup1 = soup.find_all('div', {'class': 'pubabstract'})
  list1=[]
  for abstract in soup1:
    list1.append(abstract.text)
  return list1
list2=[]
for i in range(0,100,10):
  list2=list2+forurl(i)
df=pd.DataFrame(list2)
print(df)
df.to_csv('abstract.csv')

                                                    0
0   \n                    Abstract not found\n    ...
1   \n                     describe a method for s...
2   \n                    Scaling conditional rand...
3   \n                    The paper addresses the ...
4   \n                    In most natural language...
..                                                ...
95  \n                    This paper presents a wo...
96  \n                    Abstract—Natural Languag...
97  \n                    ABSTRACT: After twenty y...
98  \n                    Text statistics are freq...
99  \n                    We summarize our experie...

[100 rows x 1 columns]


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [2]:
df.columns=["Abstracts"]
#Removing special characters and punctuations
df['Abstracts']=df['Abstracts'].str.replace('[!-;:,<>./?@#$%^&*_~{}]','')
#Remove numbers
df['Abstracts']=df['Abstracts'].str.replace('\d','')
#Remove stopwords
f=open('Stopwords.txt','r')#pulled stopwords provided
a=f.readlines()
list3=[]
for i in a:
  list3.append(i.strip())
df['Abstracts']=df['Abstracts'].apply(lambda x: " ".join(x for x in x.split() if x not in list3))
#Lower casing
df['Abstracts']=df['Abstracts'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#stemming
from nltk.stem import PorterStemmer
st=PorterStemmer()
df['Abstracts']=df['Abstracts'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#Lemmatization
import nltk
from textblob import Word
nltk.download('wordnet')
df['Abstracts']=df['Abstracts'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df.to_csv('data_clean.csv')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
from textblob import TextBlob
nltk.download('punkt')
df['Tokanization']=df['Abstracts'].apply(lambda x: TextBlob(x).words)
dic1={}
for dff in df['Tokanization']:
  op=nltk.pos_tag(dff, tagset='universal')
  dic1.update(dict(op))
print(dic1)
Noun_count=0
Verb_count=0
Adjective_count=0
Adverb_count=0
for key, value in dic1.items():
  if value=='NOUN':
    Noun_count+=1
  if value=='VERB':
    Verb_count+=1
  if value=='ADJ':
    Adjective_count+=1
  if value=='ADV':
    Adverb_count+=1
print('Total number of Nouns: {}\nTotal number of Verbs: {}\nTotal number of Adjectives: {}\nTotal number of Adverbs: {}\n'.format(Noun_count,Verb_count,Adjective_count,Adverb_count))
import spacy
nlp=spacy.load('en_core_web_sm')
for i in df['Abstracts']:
  spacy.displacy.render(nlp(i),jupyter=True)

%tensorflow_version 1.x
import benepar
import nltk
benepar.download('benepar_en3')
from benepar.spacy_plugin import BeneparComponent
nlp2= spacy.load('en')
nlp2.add_pipe(BeneparComponent('benepar_en3'))
for i in df['Abstracts']:
  j=nlp2(i)
  k=list(j.sents)[0]
  print(k._.parse_string)

list4=[]
nlp3=spacy.load('en_core_web_sm')
for i in df['Abstracts']:
  for j in nlp3(i).ents:
    list4.append((j.label_,j.text))
    print(dict(list4))
nltk.FreqDist(label for (label, text) in list4)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'abstract': 'NOUN', 'found': 'VERB', 'describ': 'VERB', 'method': 'NOUN', 'statist': 'ADJ', 'model': 'NOUN', 'base': 'VERB', 'maximum': 'ADJ', 'entropi': 'ADJ', 'we': 'PRON', 'present': 'NOUN', 'maximumlikelihood': 'VERB', 'approach': 'NOUN', 'automat': 'VERB', 'construct': 'NOUN', 'implement': 'ADJ', 'effici': 'NOUN', 'use': 'NOUN', 'exampl': 'VERB', 'sever': 'ADV', 'problem': 'NOUN', 'natur': 'ADJ', 'languag': 'ADJ', 'process': 'NOUN', 'scale': 'NOUN', 'condit': 'NOUN', 'random': 'ADJ', 'field': 'NOUN', 'term': 'NOUN', 'copyright': 'NOUN', 'work': 'NOUN', 'de

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.


/usr/local/lib/python3.7/dist-packages/benepar/spacy_plugin.py:12: FutureWarning: BeneparComponent and NonConstituentException have been moved to the benepar module. Use `from benepar import BeneparComponent, NonConstituentException` instead of benepar.spacy_plugin. The benepar.spacy_plugin namespace is deprecated and will be removed in a future version.
  FutureWarning,


(X (NP (NN abstract)) (VP (VBN found)))
(VP (VB describ) (NP (NN method)) (NP (NN statist) (NN model) (NN base)))
(NP (NP (NN scale) (RB condit) (NP (JJ random) (NN field))) (RB natur) (VB languag) (NN process) (NN term) (NN condit) (NP (NN term) (FW condit)))
(S (NP (DT the) (NN paper) (NN address)) (FW issu) (FW cooper) (NN linguist) (FW natur) (FW languag) (NN process) (FW nlp) (NN gener) (NN linguist) (NN machin) (FW translat) (FW mt) (ADJP (JJ particular)))
(SINV (PP (IN in) (NP (JJ natur) (FW languag) (NN process))) (NP (JJ applic) (NN descript) (NN logic)) (NN use) (FW encod) (NN knowledg) (NN base) (JJ syntact) (NP (NN semant) (NN pragmat) (NN element)) (NN need) (S (VP (VB drive) (JJ semant) (VB interpret) (RB natur) (FW languag) (NN gener) (NN process) (NP (ADJP (RBR more) (JJ recent)) (JJ descript) (NN logic)))))
(S (NP (PRP we)) (VP (VBP propos) (NP (FW unifi) (NML (JJ neural) (NN network)) (NN architectur) (VP (VB learn) (NN algorithm) (FW appli) (FW variou) (RB natur) (VP

FreqDist({'CARDINAL': 11,
          'DATE': 10,
          'FAC': 5,
          'GPE': 16,
          'LANGUAGE': 2,
          'LOC': 1,
          'NORP': 51,
          'ORDINAL': 4,
          'ORG': 84,
          'PERSON': 101,
          'PRODUCT': 3})

In [5]:
!pip install benepar

     |████████████████████████████████| 3.2MB 10.6MB/s 
     |████████████████████████████████| 1.9MB 37.3MB/s 
     |████████████████████████████████| 1.2MB 35.6MB/s 
     |████████████████████████████████| 890kB 34.2MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-cp37-none-any.whl size=37647 sha256=bb59084c76c9d0868246504f865551cf8e1fa0e81b91f1ec575100d7b42ce9ad
  Stored in directory: /root/.cache/pip/wheels/dd/90/28/513063023646df7774be9401c440a5f13b48bdbab15a67fc42
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=211a7e8826dae1198606f03c7bb688d61d6fb9ab44b3b5f2966becac18cb6cf5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built benepar sacremoses


In [ ]:

#3-2 Dependcy parsing
import spacy
from spacy import displacy
k=spacy.load('en_core_web_sm')
for j in covid_tweets['tweets']:
  displacy.render(k(j),jupyter=True)

#3-2 constituent parsing
%tensorflow_version 1.x
import benepar
import nltk
benepar.download('benepar_en3')
from benepar.spacy_plugin import BeneparComponent
m= spacy.load('en')
m.add_pipe(BeneparComponent('benepar_en3'))
for i in covid_tweets['tweets']:
  k= m(i)
  p = list(k.sents)[0]
  print(p._.parse_string)

#3-3 Name Entity Recognition
import spacy
from spacy import displacy
k=[]
p = spacy.load('en_core_web_sm')
for i in covid_tweets['tweets']:
  for j in p(i).ents:
    k.append((j.label_,j.text))
    print(j.label_,j.text)
nltk.FreqDist(label for (label, text) in k)

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [ ]:
'''
Sentence1: abstract found

Constituency parsing:
Constituency parsing is diving the sentence into sub-phrases, that is either Noun Phrase or Verb Phrase.
for above example, sentence1 is divided into abstract(Noun Phrase) and found(Verb Phrase)

Dependency parsing:
Dependency parsing is connecting words according to their relationships.
for above example, found is related to abstract by relation Subject
'''

'\nWrite your explanations of the constituency parsing tree and dependency parsing tree here\n\n\n\n'